In [2]:
using FFTW, Test
using CuArrays

In [ ]:
x = rand(10)
FFTW.r2r!(x, FFTW.REDFT10)

# x = rand(8, 8, 4)
# y = FFTW.r2r!(x, FFTW.REDFT10, 3)

In [ ]:
t = collect(1:16)
cat(t[1:2:N], t[N:-2:2]; dims=1)

In [ ]:
function dct_1d_gpu!(f)
    N = size(f)[1]
    f .= cat(f[1:2:N], f[N:-2:2]; dims=1)
    FFTW.fft!(f)
    f .*= 2 * exp.(collect(-1im*π*(0:N-1) / (2*N)))
    nothing
end

In [ ]:
N = 16
x = Complex.(rand(Float64, N))
y = copy(x)

dct_1d_gpu!(x)

FFTW.r2r!(y, FFTW.REDFT10);

@test real.(x) ≈ real.(y)

In [ ]:
Nx, Ny, Nz = 4, 4, 4
t = reshape(collect(1:Nx*Ny*Nz), Nx, Ny, Nz)
cat(t[:, :, 1:2:Nz], t[:, :, Nz:-2:2]; dims=3)

In [ ]:
factors = 2 * exp.(collect(-1im*π*(0:Nz-1) / (2*Nz)))
repeat(reshape(factors, 1, 1, Nz), Nx, Ny, 1)

In [3]:
function dct_dim3_gpu!(f)
    Nx, Ny, Nz = size(f)
    f .= cat(f[:, :, 1:2:Nz], f[:, :, Nz:-2:2]; dims=3)
    fft!(f, 3)

    factors = 2 * exp.(collect(-1im*π*(0:Nz-1) / (2*Nz)))
    
    # f .*= repeat(reshape(factors, 1, 1, Nz), Nx, Ny, 1)
    f .*= cu(repeat(reshape(factors, 1, 1, Nz), Nx, Ny, 1))
    
    nothing
end

dct_dim3_gpu! (generic function with 1 method)

In [5]:
Nx, Ny, Nz = 4, 4, 4
# x = Complex{Float64}.(reshape(collect(1:Nx*Ny*Nz), Nx, Ny, Nz))
x = Complex.(rand(Float64, Nx, Ny, Nz))
y = copy(x)

dct_dim3_gpu!(x)

FFTW.r2r!(y, FFTW.REDFT10, 3);

@test real.(x) ≈ real.(y)

Test Passed

In [ ]:
N = 16
t = collect(1:N)
t = cat(t[1:2:N], t[N:-2:2]; dims=1)
reshape(cat(t[1:Int(N/2)], t[end:-1:Int(N/2)+1]; dims=2)', N)

In [23]:
Nx, Ny, Nz = 4, 4, 4
t = reshape(collect(1:Nx*Ny*Nz), Nx, Ny, Nz)

4×4×4 Array{Int64,3}:
[:, :, 1] =
 1  5   9  13
 2  6  10  14
 3  7  11  15
 4  8  12  16

[:, :, 2] =
 17  21  25  29
 18  22  26  30
 19  23  27  31
 20  24  28  32

[:, :, 3] =
 33  37  41  45
 34  38  42  46
 35  39  43  47
 36  40  44  48

[:, :, 4] =
 49  53  57  61
 50  54  58  62
 51  55  59  63
 52  56  60  64

In [24]:
t = cat(t[:, :, 1:2:Nz], t[:, :, Nz:-2:2]; dims=3)
t = cat(t[:, :, 1:Int(Nz/2)], t[:, :, end:-1:Int(Nz/2)+1]; dims=4)
reshape(permutedims(t, (1, 2, 4, 3)), Nx, Ny, Nz)

4×4×4 Array{Int64,3}:
[:, :, 1] =
 1  5   9  13
 2  6  10  14
 3  7  11  15
 4  8  12  16

[:, :, 2] =
 17  21  25  29
 18  22  26  30
 19  23  27  31
 20  24  28  32

[:, :, 3] =
 33  37  41  45
 34  38  42  46
 35  39  43  47
 36  40  44  48

[:, :, 4] =
 49  53  57  61
 50  54  58  62
 51  55  59  63
 52  56  60  64

In [4]:
function idct_dim3_gpu!(f)
    Nx, Ny, Nz = size(f)
    
    bfactors = 0.5 * exp.(collect(1im*π*(0:Nz-1) / (2*Nz)))
    # f .*= repeat(reshape(bfactors, 1, 1, Nz), Nx, Ny, 1)
    f .*= cu(repeat(reshape(bfactors, 1, 1, Nz), Nx, Ny, 1))
    
    ifft!(f, 3)
    
    # f = cat(f[:, :, 1:Int(Nz/2)], f[:, :, end:-1:Int(Nz/2)+1]; dims=4)
    # f = reshape(permutedims(f, (1, 2, 4, 3)), Nx, Ny, Nz)
    # f .= reshape(permutedims(cat(f[:, :, 1:Int(Nz/2)], f[:, :, end:-1:Int(Nz/2)+1]; dims=4), (1, 2, 4, 3)), Nx, Ny, Nz)
    f .= cu(reshape(permutedims(cat(f[:, :, 1:Int(Nz/2)], f[:, :, end:-1:Int(Nz/2)+1]; dims=4), (1, 2, 4, 3)), Nx, Ny, Nz))
    
    nothing
end

idct_dim3_gpu! (generic function with 1 method)

In [8]:
Nx, Ny, Nz = 16, 8, 4
x = Complex{Float64}.(reshape(collect(1:Nx*Ny*Nz), Nx, Ny, Nz))
y = copy(x)

dct_dim3_gpu!(x)
FFTW.r2r!(y, FFTW.REDFT10, 3);

@show @test real.(x) ≈ real.(y)

idct_dim3_gpu!(x)
FFTW.r2r!(y, FFTW.REDFT01, 3);
@. y /= 2Nz;

@show @test real.(x) ≈ real.(y);

#= In[8]:8 =# @test(real.(x) ≈ real.(y)) = Test Passed
#= In[8]:14 =# @test(real.(x) ≈ real.(y)) = Test Passed


In [52]:
real.(x)

4×4×4 Array{Float64,3}:
[:, :, 1] =
 1.0  5.0   9.0  13.0
 2.0  6.0  10.0  14.0
 3.0  7.0  11.0  15.0
 4.0  8.0  12.0  16.0

[:, :, 2] =
 17.0  21.0  25.0  29.0
 18.0  22.0  26.0  30.0
 19.0  23.0  27.0  31.0
 20.0  24.0  28.0  32.0

[:, :, 3] =
 33.0  37.0  41.0  45.0
 34.0  38.0  42.0  46.0
 35.0  39.0  43.0  47.0
 36.0  40.0  44.0  48.0

[:, :, 4] =
 49.0  53.0  57.0  61.0
 50.0  54.0  58.0  62.0
 51.0  55.0  59.0  63.0
 52.0  56.0  60.0  64.0

In [53]:
real.(y)

4×4×4 Array{Float64,3}:
[:, :, 1] =
 1.0  5.0   9.0  13.0
 2.0  6.0  10.0  14.0
 3.0  7.0  11.0  15.0
 4.0  8.0  12.0  16.0

[:, :, 2] =
 17.0  21.0  25.0  29.0
 18.0  22.0  26.0  30.0
 19.0  23.0  27.0  31.0
 20.0  24.0  28.0  32.0

[:, :, 3] =
 33.0  37.0  41.0  45.0
 34.0  38.0  42.0  46.0
 35.0  39.0  43.0  47.0
 36.0  40.0  44.0  48.0

[:, :, 4] =
 49.0  53.0  57.0  61.0
 50.0  54.0  58.0  62.0
 51.0  55.0  59.0  63.0
 52.0  56.0  60.0  64.0

In [54]:
@. real(x) / real(y)

4×4×4 Array{Float64,3}:
[:, :, 1] =
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0

[:, :, 2] =
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0

[:, :, 3] =
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0

[:, :, 4] =
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0

In [13]:
Nx, Ny, Nz = 16, 8, 4
x = Complex{Float64}.(reshape(collect(1:Nx*Ny*Nz), Nx, Ny, Nz))
y = copy(x)

dct_dim3_gpu!(x)
FFTW.r2r!(y, FFTW.REDFT10, 3);
@show @test real.(x) ≈ real.(y)

FFTW.fft!(x, [1, 2])
FFTW.fft!(y, [1, 2])

FFTW.ifft!(x, [1, 2])
FFTW.ifft!(y, [1, 2])

idct_dim3_gpu!(x)
FFTW.r2r!(y, FFTW.REDFT01, 3);
@. y /= 2Nz;

@show @test real.(x) ≈ real.(y);

#= In[13]:7 =# @test(real.(x) ≈ real.(y)) = Test Passed
#= In[13]:19 =# @test(real.(x) ≈ real.(y)) = Test Passed
